In [2]:
import sys
import pandas as pd
import numpy as np
import re
import warnings
import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder

In [16]:
def isnat(x):
    return x.astype('i8') == np.datetime64('NaT').astype('i8')

def clean_white_space(data):
    
    for i in range(len(data)):
        ind = 0 
        for g in data.iloc[i] :
            try:
                g = g.split()
                if g == []:
                    data.iloc[i,ind]= np.nan
                ind = ind +1
            except:
                ind = ind +1
                pass

    return (data)

def isfloat(value):
    try:
        float(value)
        return True
    
    except ValueError:
        return False

def check_types(new_data):
    
    marks = {}.fromkeys(['to_numeric', 'to_string','null','boolean','to_datetime'], 0)

    for index in range(len(new_data)):
        
        row = new_data[index]
        
        if ((isfloat(row)) &  (pd.isna(row)is False)):
             row=float(row)
             new_data[index]=row
             marks['to_numeric'] += 1
                          
        if isinstance(row,str):
            if (((row.lower() == 'true') | (row.lower() == 'false'))is False):
                    try:
                        new_data[index]=pd.to_datetime(row)
                        if isinstance(row, pd._libs.tslibs.nattype.NaTType):
                            print('row')
                            new_data[index] = np.nan()
                        marks['to_datetime'] += 1
                
                    except:
                          marks['to_string'] += 1
                     
            elif (row.lower() == 'true') | (row.lower() == 'false'):
                     marks['boolean'] += 1                  
                
        if pd.isna(row):
                marks['null'] += 1            
                        
    print(marks)
    return(marks)

def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset

    return(dataset)

def DataCleaning(data):
    # בדיקה אם יש white space
    data = clean_white_space(data)
    # בדיקה אם יש עמודה רק של ערכי null והורדה של אחת כזו
    cols_to_check = data.columns
    data['is_na'] = data[cols_to_check].isnull().apply(lambda x: all(x), axis=1) 
    data = data[data.is_na==False]
    data=data.drop('is_na',axis = 1)
    # הורדה של שורות אותו דבר
    data = data.drop_duplicates()
    nunique = data.nunique()
    cols_to_drop = nunique[nunique == 1].index
    data = data.drop(cols_to_drop, axis=1)
    data=data[data.columns[data.isnull().mean() < 0.80]]
    data=data.reset_index()
    data=data.drop('index',axis = 1)

    num_to_impute = []
    not_drop = []
    for i in data:
        new_data = data.loc[:,i]
        marks = check_types(new_data)
        warnings.filterwarnings('ignore')
        max_key = max(marks, key=marks.get)

        if max_key == 'to_numeric':
            data.loc[:,i]=pd.to_numeric(data.loc[:,i], errors='coerce')

            if new_data.isnull().mean() >= 0.8:
                data=data.drop(i , axis = 1)

            else:
                num_to_impute.append(i)

        if max_key == 'to_datetime':
            data.loc[:,i]=pd.to_datetime(data.loc[:,i], errors='coerce')

            if (1-(new_data.value_counts().sum()/len(new_data))) >= 0.8:
                data=data.drop(i , axis = 1)
            else:
                data.loc[:,i] = data.loc[:,i].fillna(data.loc[:,i].value_counts().index[0])
                not_drop.append(i)
        elif   max_key == 'to_string': 
            data.loc[:,i].to_string()
            data.loc[:,i] = data.loc[:,i].fillna(data.loc[:,i].value_counts().index[0])

        if  max_key == 'boolean':
            data.loc[:,i]=data.loc[:,i].astype('bool')

            if new_data.isnull().mean() >= 0.8:
                data=data.drop(i , axis = 1)
            else:
                data.loc[:,i] = data.loc[:,i].fillna(data.loc[:,i].value_counts().index[0])
                not_drop.append(i)
        if (new_data.nunique()/len(new_data))<=0.14:
            data.loc[:,i]=data.loc[:,i].astype("category")
            data.loc[:,i] = data.loc[:,i].fillna(data.loc[:,i].value_counts().index[0])
        print(max_key)
    data

    imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
    imputer = imputer.fit(data.loc[:,num_to_impute])
    data.loc[:,num_to_impute] = imputer.transform(data.loc[:,num_to_impute])
    scaler = MinMaxScaler()
    data.loc[:,num_to_impute] = scaler.fit_transform(data.loc[:,num_to_impute])
    y=data.loc[:,not_drop]
    X=data.drop(num_to_impute, axis = 1)
    df1=pd.get_dummies(X,drop_first=True)
    data=pd.concat([df1,data],axis=1)
    new_to_drop = []
    data.drop(X.columns,axis=1,inplace=True)
    data=pd.concat([y,data],axis=1)
    data = correlation(data,0.9)
    return(data)